In [1]:
import importlib
import os
from datetime import datetime

import numpy as np
import pandas as pd
import yaml

In [ ]:
DATA_PATH = 'draft_1.xlsx'
SHEET_NAME = None  # set to a sheet name to override
TOLERANCE = 0.02

# Config source options:
CONFIG_YAML = None  # set a YAML string to override
CONFIG_PATH = None  # set a path to a YAML file to override

print('DATA_PATH:', DATA_PATH)
print('SHEET_NAME:', SHEET_NAME)
print('TOLERANCE:', TOLERANCE)

DATA_PATH: draft_1.xlsx
SHEET_NAME: None
TOLERANCE: 0.02
USE_PROJECT_SAMPLE: True


In [3]:
# ---- Load Excel ----
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f'File not found: {DATA_PATH}')

try:
    xls = pd.ExcelFile(DATA_PATH)
except ImportError as exc:
    raise ImportError('Missing dependency openpyxl. Install with: pip install openpyxl') from exc

print('Sheets:', xls.sheet_names)
sheet_to_use = SHEET_NAME or xls.sheet_names[0]
df = pd.read_excel(DATA_PATH, sheet_name=sheet_to_use)
print('Loaded sheet:', sheet_to_use)
df.head()


Sheets: ['Sheet1']
Loaded sheet: Sheet1


,Unnamed: 0,loyalty,discount,mobile_app,online,high_spend,subscription,return,promo_email,customer_support_contact,churn_risk
0,0,1,1,1,0,0,0,0,1,0,0
1,1,1,1,0,0,0,1,0,1,0,0
2,2,0,0,0,0,0,1,0,1,0,0
3,3,1,0,0,0,1,0,0,0,0,0
4,4,1,0,1,0,0,0,0,0,0,0


In [7]:
print('Shape:', df.shape)
display(df.dtypes)


Shape: (1000, 11)


Unnamed: 0                  int64
loyalty                     int64
discount                    int64
mobile_app                  int64
online                      int64
high_spend                  int64
subscription                int64
return                      int64
promo_email                 int64
customer_support_contact    int64
churn_risk                  int64
dtype: object

In [8]:
def is_binary(series):
    values = set(series.dropna().unique().tolist())
    allowed = {0, 1, True, False}
    return values.issubset(allowed) and len(values) > 0

binary_cols = [c for c in df.columns if is_binary(df[c])]
print('Binary columns:', binary_cols)

binary_rates = (df[binary_cols] == 1).mean().sort_values(ascending=False)
display(binary_rates)


Binary columns: ['loyalty', 'discount', 'mobile_app', 'online', 'high_spend', 'subscription', 'return', 'promo_email', 'customer_support_contact', 'churn_risk']


mobile_app                  0.460
churn_risk                  0.411
loyalty                     0.356
subscription                0.296
promo_email                 0.289
high_spend                  0.279
discount                    0.199
customer_support_contact    0.139
online                      0.074
return                      0.051
dtype: float64

In [9]:
CONFIG_1 = """
metadata:
  name: "simple_demo_extended"
  version: "1.1"

columns:
  - column_id: "loyalty"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "bernoulli"
      probabilities: { true_prob: 0.35, false_prob: 0.65 }

  - column_id: "discount"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "bernoulli"
      probabilities: { true_prob: 0.20, false_prob: 0.80 }

  - column_id: "online"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["discount"]
      conditional_probs:
        "discount=1": { true_prob: 0.12, false_prob: 0.88 }
        "discount=0": { true_prob: 0.07, false_prob: 0.93 }

  - column_id: "return"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["online", "loyalty"]
      conditional_probs:
        "online=1, loyalty=1": { true_prob: 0.12, false_prob: 0.88 }
        "online=1, loyalty=0": { true_prob: 0.05, false_prob: 0.95 }
        "online=0, loyalty=1": { true_prob: 0.07, false_prob: 0.93 }
        "online=0, loyalty=0": { true_prob: 0.04, false_prob: 0.96 }

  - column_id: "mobile_app"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "bernoulli"
      probabilities: { true_prob: 0.45, false_prob: 0.55 }

  - column_id: "high_spend"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["loyalty", "discount"]
      conditional_probs:
        "loyalty=1, discount=1": { true_prob: 0.55, false_prob: 0.45 }
        "loyalty=1, discount=0": { true_prob: 0.45, false_prob: 0.55 }
        "loyalty=0, discount=1": { true_prob: 0.25, false_prob: 0.75 }
        "loyalty=0, discount=0": { true_prob: 0.15, false_prob: 0.85 }

  - column_id: "customer_support_contact"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["return", "high_spend", "online"]
      conditional_probs:
        "return=1, high_spend=1, online=1": { true_prob: 0.65, false_prob: 0.35 }
        "return=1, high_spend=1, online=0": { true_prob: 0.55, false_prob: 0.45 }
        "return=1, high_spend=0, online=1": { true_prob: 0.45, false_prob: 0.55 }
        "return=1, high_spend=0, online=0": { true_prob: 0.35, false_prob: 0.65 }
        "return=0, high_spend=1, online=1": { true_prob: 0.25, false_prob: 0.75 }
        "return=0, high_spend=1, online=0": { true_prob: 0.20, false_prob: 0.80 }
        "return=0, high_spend=0, online=1": { true_prob: 0.15, false_prob: 0.85 }
        "return=0, high_spend=0, online=0": { true_prob: 0.10, false_prob: 0.90 }

  - column_id: "subscription"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["loyalty", "mobile_app"]
      conditional_probs:
        "loyalty=1, mobile_app=1": { true_prob: 0.60, false_prob: 0.40 }
        "loyalty=1, mobile_app=0": { true_prob: 0.45, false_prob: 0.55 }
        "loyalty=0, mobile_app=1": { true_prob: 0.25, false_prob: 0.75 }
        "loyalty=0, mobile_app=0": { true_prob: 0.10, false_prob: 0.90 }

  - column_id: "promo_email"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on: ["discount", "subscription"]
      conditional_probs:
        "discount=1, subscription=1": { true_prob: 0.75, false_prob: 0.25 }
        "discount=1, subscription=0": { true_prob: 0.55, false_prob: 0.45 }
        "discount=0, subscription=1": { true_prob: 0.35, false_prob: 0.65 }
        "discount=0, subscription=0": { true_prob: 0.15, false_prob: 0.85 }

  - column_id: "churn_risk"
    values: { true_value: 1, false_value: 0 }
    distribution:
      type: "conditional"
      depend_on:
        ["loyalty", "return", "customer_support_contact", "subscription", "high_spend"]
      conditional_probs:
        "loyalty=0, return=1, customer_support_contact=1, subscription=0, high_spend=0":
          { true_prob: 0.85, false_prob: 0.15 }
        "loyalty=0, return=1, customer_support_contact=0, subscription=0, high_spend=0":
          { true_prob: 0.70, false_prob: 0.30 }
        "loyalty=1, return=1, customer_support_contact=1, subscription=1, high_spend=1":
          { true_prob: 0.25, false_prob: 0.75 }
        "loyalty=1, return=0, customer_support_contact=0, subscription=1, high_spend=1":
          { true_prob: 0.10, false_prob: 0.90 }
        "loyalty=1, return=0, customer_support_contact=0, subscription=0, high_spend=0":
          { true_prob: 0.20, false_prob: 0.80 }
"""

In [10]:
config = None

CONFIG_YAML = CONFIG_1

if CONFIG_YAML:
    config = yaml.safe_load(CONFIG_YAML)
elif CONFIG_PATH:
    with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f.read())

config is not None


True

In [11]:
def parse_condition(key):
    parts = str(key).split(',')
    result = {}
    for part in parts:
        if '=' not in part:
            continue
        k, v = part.strip().split('=', 1)
        result[k.strip()] = int(v.strip())
    return result

def check_marginals(df, config):
    rows = []
    for col in config.get('columns', []):
        col_id = col.get('column_id')
        dist = col.get('distribution', {})
        if dist.get('type') != 'bernoulli':
            continue
        target = float(dist['probabilities']['true_prob'])
        if col_id not in df.columns:
            rows.append({
                'column_id': col_id,
                'target': target,
                'observed': None,
                'error': None,
                'pass': False,
                'note': 'missing column'
            })
            continue
        observed = float((df[col_id] == 1).mean())
        error = abs(observed - target)
        rows.append({
            'column_id': col_id,
            'target': target,
            'observed': observed,
            'error': error,
            'pass': error <= TOLERANCE,
            'note': ''
        })
    return pd.DataFrame(rows)

def check_conditionals(df, config):
    rows = []
    for col in config.get('columns', []):
        col_id = col.get('column_id')
        dist = col.get('distribution', {})
        if dist.get('type') != 'conditional':
            continue
        cond_probs = dist.get('conditional_probs', {})
        for cond_key, probs in cond_probs.items():
            if not isinstance(probs, dict) or probs.get('true_prob') is None:
                continue
            target = float(probs['true_prob'])
            cond_map = parse_condition(cond_key)
            if col_id not in df.columns:
                rows.append({
                    'column_id': col_id,
                    'condition': cond_key,
                    'target': target,
                    'observed': None,
                    'n_group': 0,
                    'error': None,
                    'pass': False,
                    'note': 'missing column'
                })
                continue
            mask = np.ones(len(df), dtype=bool)
            missing = False
            for k, v in cond_map.items():
                if k not in df.columns:
                    missing = True
                    break
                mask &= (df[k] == v).values
            if missing:
                rows.append({
                    'column_id': col_id,
                    'condition': cond_key,
                    'target': target,
                    'observed': None,
                    'n_group': 0,
                    'error': None,
                    'pass': False,
                    'note': 'missing dependency'
                })
                continue
            n_group = int(mask.sum())
            if n_group == 0:
                rows.append({
                    'column_id': col_id,
                    'condition': cond_key,
                    'target': target,
                    'observed': None,
                    'n_group': 0,
                    'error': None,
                    'pass': False,
                    'note': 'empty group'
                })
                continue
            observed = float((df[col_id].values[mask] == 1).mean())
            error = abs(observed - target)
            rows.append({
                'column_id': col_id,
                'condition': cond_key,
                'target': target,
                'observed': observed,
                'n_group': n_group,
                'error': error,
                'pass': error <= TOLERANCE,
                'note': ''
            })
    return pd.DataFrame(rows)

if config is None:
    print('No config loaded. Only EDA summary is available.')
else:
    marginal_report = check_marginals(df, config)
    conditional_report = check_conditionals(df, config)
    display(marginal_report)
    display(conditional_report)


,column_id,target,observed,error,pass,note
0,loyalty,0.35,0.356,0.006,True,
1,discount,0.20,0.199,0.001,True,
2,mobile_app,0.45,0.460,0.010,True,


,column_id,condition,target,observed,n_group,error,pass,note
0,online,discount=1,0.12,0.125628,199,0.005628,True,
1,online,discount=0,0.07,0.061174,801,0.008826,True,
2,return,"online=1, loyalty=1",0.12,0.120000,25,0.000000,True,
3,return,"online=1, loyalty=0",0.05,0.040816,49,0.009184,True,
4,return,"online=0, loyalty=1",0.07,0.066465,331,0.003535,True,
5,return,"online=0, loyalty=0",0.04,0.040336,595,0.000336,True,
6,high_spend,"loyalty=1, discount=1",0.55,0.551282,78,0.001282,True,
7,high_spend,"loyalty=1, discount=0",0.45,0.467626,278,0.017626,True,
8,high_spend,"loyalty=0, discount=1",0.25,0.247934,121,0.002066,True,
9,high_spend,"loyalty=0, discount=0",0.15,0.145315,523,0.004685,True,
